In [ ]:
# Optional: mount Google Drive to persist data between sessions
# from google.colab import drive
# drive.mount('/content/drive')


In [ ]:
# Install quran-segmenter (editable) and run the one-shot Colab setup.
import pathlib
if not pathlib.Path('/content/quran_segmenter').exists():
    !git clone https://github.com/ashaltu/quran_segmenter.git /content/quran_segmenter

!cd /content/quran_segmenter && pip install -q -e .

WORDS="/content/qpc-hafs-word-by-word.json"
METADATA="/content/quran-metadata-misc.json"
CONFIG="/content/quran_data/config.json"

!quran-segmenter setup-colab \
  --words "$WORDS" \
  --metadata "$METADATA" \
  --config "$CONFIG"


In [ ]:
# Init data dir and register your translation.
# Default: English demo (pre-segmented) using provided example + downloaded embeddings.
TRANSLATION_ID="en-sahih-international-simple"
TRANSLATION_FILE="/content/quran_segmenter/example/en-sahih-international-simple.json"
TRANSLATION_NAME="Sahih International (simple)"
LANGUAGE_CODE="en"

!quran-segmenter init --data-dir ./quran_data

# (Optional) If you already have embeddings, copy them from Drive.
# !cp -rf /content/drive/MyDrive/QuranSegmenter/embeddings/* ./quran_data/embeddings/.

# (Optional) Download spans.npz and English embeddings (contains en-sahih-international-simple.npz).
# !cd /content/quran_data/embeddings && git clone -q https://huggingface.co/datasets/rehandaphedar/rabtize .

!quran-segmenter register "$TRANSLATION_ID" "$TRANSLATION_FILE" \
  --name "$TRANSLATION_NAME" \
  --language "$LANGUAGE_CODE" \
  --spans-embeddings-filepath /content/quran_data/embeddings/spans.npz \
  --segment-embeddings-filepath /content/quran_data/embeddings/${TRANSLATION_ID}.npz


In [ ]:
# Prepare translation (requires GEMINI_API_KEY unless using pre-segmented + downloaded embeddings).
# %env GEMINI_API_KEY=your_key
!quran-segmenter prepare "$TRANSLATION_ID"


In [ ]:
# Process audio
AUDIO_FILE="19.mp3"
VERSE_SPEC="19"  # e.g., "taawwudh+basmalah+19" or "2:1-5"

!quran-segmenter process \
  "$AUDIO_FILE" \
  "$VERSE_SPEC" \
  "$TRANSLATION_ID" \
  -o output.json \
  --start-server
